# Install Conda Environments in OpenSARlab using an environment.yml

- Conda environments are isolated Python software environments. 

- They allow you to install and use multiple software configurations that would conflict with each other in a single environment.   

In [ ]:
import fileinput
import ipywidgets
import os
from pathlib import Path

from IPython.display import Markdown, display, HTML

### List Currently Installed Conda Environments

In [ ]:
!mamba env list

---
### Build an ASF Conda Environment or Create Your Own

- ASF data recipe environments are defined in `yaml` files in the `~/conda_environments/Environment_Configs` directory

**Note: to be seen and built using this notebook, new environment.yml files should be:**
- stored in /home/jovyan/Environment_Configs
- named so that they end in `_env.yml` or `_env.yaml`

### Run the code cell below for a link to an example minimal environment for OpenSARLab
- supports ipywidgets and interactive matplotlib plots

In [ ]:
yaml_path = Path.cwd()/'Environment_Configs/minimal_notebook_env.yaml'
relative_path = yaml_path.relative_to(Path.cwd())    
link = f"<a href='{relative_path}'>{yaml_path}</a>"
html = HTML(link)
display(html)

---
### Select a Conda Environment to Create

In [ ]:
env_ymls = list(Path.cwd().rglob("Environment_Configs/*.y*ml"))

yml_dict = {fileinput.filename():f.split('name: ')[1].strip('\n') for f in fileinput.input(env_ymls) if fileinput.isfirstline()}
names = [v for v in yml_dict.values()]
if len(names) > len(set(names)):
    dups = {}
    
    for k, v in yml_dict.items():
        dups.setdefault(v, []).append(k)
    dups = {k:v for (k,v) in dups.items() if len(dups[k]) > 1}
    display(Markdown(f'<text style=color:red><strong>WARNING:</text>'))
    display(Markdown(f'<text style=color:red>Duplicate environment names found in /home/jovyan/conda_environments/Environment_Configs/*.yml files.</text>'))
    display(Markdown(f'<text style=color:red>Creating an env with the same name as an existing env will overwrite the original.</text>'))
    display(Markdown(f'<text style=color:green><br><strong>RECOMMENDATION:</strong></text>'))
    display(Markdown(f'<text style=color:green>Edit the environment names on the first lines of the YMLs to eliminate duplicates.</text>'))
    display(Markdown(f'<text style=color:green>If you see multiple similarly named YMLs, some of whose filenames have a timestamp appended to them, this is a result of YML updates.</text>'))
    display(Markdown(f'<text style=color:green>In this case, the YML without a timestamp is the latest version and, unless you have made customizations to the old versions, you may safely delete them.</text>'))
    
    for d in dups:
        print(f'YMLs using the environment name "{d}":')
        for pth in dups[d]:
            print(pth)
    options = [f"{yml_dict[k]} ---> {k}" for k in yml_dict]
else:
    options = [yml_dict[k] for k in yml_dict]
options.sort()
display(Markdown(f'<text style=color:green><br><strong>Select an Environment to Create:</strong></text>'))
height = len(env_ymls) * 20
names =  ipywidgets.RadioButtons(
    options=options,
    description="",
    disabled=False,
    layout=ipywidgets.Layout(height=f"{height}px", width='1000px')
)
names


---
### Collect the Environment Name, Prefix, and Path to its env.yml

In [ ]:
name = names.value
if '--->' in name:
    name, yml = name.split(' ---> ')
else:
    yml = {v:k for k,v in yml_dict.items()}[name]
prefix = f"/home/jovyan/.local/envs/{name}"

---
### Create the Environment, change its display name, and run any needed shell scripts
- This will overwrite an existing environment with the same name instead of updating it, which is the safer option for environments that change PATH and/or PYTHONPATH
- custom shell scripts 
    - save in `~/conda_environments/Scripts`
    - append an `elif` statement to the `if`/`elif` statements below to run it

In [ ]:
# Creates the environment
!mamba env create -f $yml --prefix $prefix -q --force

# Without running this, every environment would appear in the GUI with the same name, 'Python 3'
!mamba run -n $name kernda --display-name $name -o $prefix/share/jupyter/kernels/python3/kernel.json

# Runs any associated shell scripts
if name == "insar_analysis":
    !source "Scripts/install_insar_analysis_pkgs.sh"
elif name == "train":
    !source "Scripts/train.sh"
elif name == "earthscope_insar":
    !source "Scripts/earthscope_insar.sh"
elif name == "NISAR_SE":
    !source "Scripts/nisar_se.sh"
elif name == "isce3_rtc":
    !source "Scripts/isce3_rtc.sh"

---
### Remove unused packages and caches.

In [ ]:
!mamba clean -p -t --yes
!pip cache purge

*Create_OSL_Conda_Environments.ipynb - Version 1.2.1 - March 2023*

*Version Changes:*
- *add instructions for creating custom `environment.yaml` files and shell scripts*
- *move shell scripts to `Scripts` directory*